In [7]:
import requests
import lxml.html
from lxml import etree

In [3]:
r = requests.get("https://leetcode.com/discuss/questions/oj/guess-number-higher-or-lower")

In [6]:
print r.text

<!DOCTYPE html>
<html lang="en-US">
<head>
	<title>Guess Number Higher or Lower | LeetCode Discuss</title>
	<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no" />
	<meta name="content-type" content="text/html; charset=UTF-8" />
	<meta name="apple-mobile-web-app-capable" content="yes" />
	<meta name="mobile-web-app-capable" content="yes" />
	<meta property="og:site_name" content="LeetCode Discuss" />
	<meta name="keywords" />
	<meta name="msapplication-badge" content="frequency=30; polling-uri=https://discuss.leetcode.com/sitemap.xml" />
	<meta name="msapplication-square150x150logo" content="/uploads/system/site-logo.png" />
	<meta name="title" content="Guess Number Higher or Lower" />
	<meta property="og:title" content="Guess Number Higher or Lower" />
	<meta name="description" />
	<meta property="og:type" content="website" />
	<link rel="stylesheet" type="text/css" href="/stylesheet.css?96ccbe42-bea4-4379-bfd5-50968f7408f2" />
	
	<link rel="icon" ty

In [9]:
def get_doc_root(url):
    r = requests.get(url)
    html_text = r.text
    my_parser = etree.HTMLParser(encoding='utf-8')
    root = etree.HTML(html_text, parser=my_parser)
    return root

In [10]:
url = "https://leetcode.com/discuss/questions/oj/guess-number-higher-or-lower"

In [11]:
root = get_doc_root(url)

In [12]:
items = root.xpath('//h2[@class="title"]')

In [13]:
len(items)

20

In [22]:
for item in items:
    title = item.xpath('./a')[0]
    print title.text

My C++ solution using result of guess()
Short Java code using binary search
Any problem with the test case?
Python iterative solution
There is a trap in the question.
1ms Java Binary Search Solution
java binary search
Simple java binary search solution
Why not support JavaScript?
For people who suffered from binary search implementation details
Easy binary search solution in Python
C++ Standard Binary Search
Why is this problem on the site?
one BinarySearch template to AC all the related problem
Java Solution using Binary Search (Iterative)
Why my binary search get time limit exceed error
The key point is to read the problem carefully.
Easy Java Solution using Binary Search. Type casting works faster then if/else statement.
C++ simple recursive 0ms, 3 main lines
Fastest C++Solution considering all situations


In [23]:
import feedparser

In [24]:
r = "https://discuss.leetcode.com/category/492.rss"

In [25]:
d = feedparser.parse(r)

In [27]:
d.keys()

['feed',
 'status',
 'version',
 'encoding',
 'bozo',
 'headers',
 'href',
 'etag',
 'namespaces',
 'entries']

In [29]:
len(d["entries"])

26